# Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
from pathlib import Path

from PIL import Image
from google import genai

import tt

LOG = logging.getLogger(__name__)
tt.logging_init()

In [ ]:
CLASSES = [
    "chicken",
    "cow",
    "creeper",
    "enderman",
    "pig",
    "sheep",
    "skeleton",
    "spider",
    "villager",
    "zombie",
]
CLASSES.sort()
CLASSES_MINECRAFT = [f"minecraft {x}" for x in CLASSES]

flash = "gemini-2.5-flash"
flash_lite = "gemini-2.5-flash-lite"
prompt_short = f"""Detect {", ".join(CLASSES)}. Return AT MOST 20 detections."""

mobs_path = Path.home() / "src/data/mobs1/640"

QCFG = tt.GeminiQueryConfig(prompt_short, CLASSES)

# File upload

In [ ]:
GF = tt.GeminiFileAPI()
# GF.upload_dir(mobs_path)
GF.sync()
print(GF.gfiles)

# 

# Detect

In [ ]:
# image640 = Image.open(mobs_path / "06092b21-2024-10-20_22.22.09.png")
# image640 = Image.open(sorted(mobs_path.glob("*.png"))[5])
# This image causes gemini-2.5-flash-lite to detect the pig repeatedly.
image_break = "/Users/joe/src/data/mobs1/640/cda62cfe-2024-10-20_22.22.31.png"
image640 = Image.open(image_break)
# display(image640)
bbox = tt.gemini_gen_bboxes(image640, QCFG)
print(bbox)
display(tt.plot_bb(image640, bbox, CLASSES))

In [ ]:
# Display results of one query at a time
def detect_single(gfile: genai.types.File):
    return tt.gemini_detect_gfile(gfile, QCFG)
tt.InferViewer[genai.types.File](detect_single, GF.gfiles).show_widget()

In [ ]:
# Run all queries first, then display the results
GFILES10 = GF.gfiles[:10]
results = tt.gemini_detect_multi(GFILES10, QCFG)
tt.InferViewer[tt.ImageResult](lambda x: x, results).show_widget()